### 示例12-1

In [ ]:
import folium
from folium import FeatureGroup
from folium import CircleMarker
input_file = open(r'c:\data\taxi\20150420_26843.txt','r')
taxi_layer = FeatureGroup(name='出租车轨迹点', control=True)
for record in input_file:
    lng = float(record.split(",")[8])
    lat = float(record.split(",")[9])
    CircleMarker(location = (lat,lng),
               radius=1,
               weight=2,).add_to(taxi_layer)
input_file.close
bounds = taxi_layer.get_bounds()
tiles = "https://{s}.tile.openstreetmap.de/{z}/{x}/{y}.png"
attr = "OpenStreetMap.DE"
m = folium.Map(tiles=tiles,
               attr=attr,
               width=600,
               height=400,
               zoom_start=12)
m.fit_bounds(bounds)
taxi_layer.add_to(m)
folium.LayerControl().add_to(m)
m

### 示例12-2

In [ ]:
import folium
from folium.plugins import TimestampedGeoJson
from geojson import LineString,Feature,FeatureCollection

input_file = open(r'c:\data\taxi\20150420_26843.txt','r')
coordinates = []
times = []
for record in input_file: 
    time = record.split(",")[6]
    times.append(time)
    lng = float(record.split(",")[8])
    lat = float(record.split(",")[9])
    coordinates.append([lng,lat])

lineString = LineString(coordinates)
feature = Feature(geometry=lineString,
               properties={"times":times})
start_point = lineString["coordinates"][0]  #作为地图中心点
m = folium.Map(location=[start_point[1],start_point[0]],
              width = "80%",
              height = "80%",
              zoom_start=10)
TimestampedGeoJson(data=feature,
                   period="PT10S",   
                   add_last_point=True).add_to(m)
m

### 示例12-3

In [ ]:
import pandas as pd
i = 0
on_offs = []   #存储上下客点
input_file = open(r'c:\data\taxi\20150420_26843.txt','r')
#提取上下客点
for record in input_file:
    i = i + 1
    is_empty = record.split(",")[2]        #是否空车 
    time = record.split(",")[6]
    lng = float(record.split(",")[8])
    lat = float(record.split(",")[9])
    if i == 1:
        former_empty = is_empty
    else:
        if former_empty == "1" and is_empty == "0":
            on_offs.append([1,time,lng,lat])
        elif former_empty == "0" and is_empty == "1":
            on_offs.append([0,time,lng,lat])
        former_empty = is_empty
input_file.close

#去除第一个点是下客点以及最后一个点是上客点
if on_offs[0][0] == 0:
    on_offs.pop(0)
if on_offs[-1][0] == 1:
    on_offs.pop(-1)

def second_hms(second):
    hour = int(second/3600)
    minute = int((second - 3600*hour)/60)
    second = int(second - 3600*hour - 60*minute)
    return str(hour) + ":" + str(minute) + ":" + str(second)

def hms_second(hms):
    hour,minute,second = hms.split(":")
    time_s = int(hour)*3600+int(minute)*60+int(second)
    return time_s

#利用百度地图逆地理编码服务返回语义地址信息
def place (lng,lat):
    import requests
    import json
    service = "http://api.map.baidu.com/reverse_geocoding/v3/?"
    output = "json"
    AK= "申请的密钥" 
    coordtype = "wgs84ll"
    parameters = f"location={lat},{lng}&output={output}&\
    coordtype={coordtype}&extensions_poi=1&ak={AK}"
    url = service + parameters
    response = requests.get(url)
    text = response.text
    dic = json.loads(text)
    status = dic["status"]
    if status == 0:
        sematic_description = dic["result"]["sematic_description"]
        return sematic_description
    else:
        print("逆向地理编码不成功")      

on_time_list = []
on_places = []
off_time_list = []
off_places = []
durations = []
for i in range(0,len(on_offs),2):
    on_time = on_offs[i][1].split(" ")[1]
    on_time_list.append(on_time)
    on_place = place(on_offs[i][2],on_offs[i][3])
    on_places.append(on_place)
    off_time = on_offs[i+1][1].split(" ")[1]
    off_time_list.append(off_time)
    off_place = place(on_offs[i+1][2],on_offs[i+1][3])
    off_places.append(off_place)
    duration = hms_second(off_time) - hms_second(on_time)
    durations.append(second_hms(duration))
frame = pd.DataFrame()
frame["上车地点"] = on_places
frame["上车时间"] = on_time_list
frame["下车地点"] = off_places
frame["下车时间"] = off_time_list
frame["持续时间"] = durations
frame

### 示例12-4

In [ ]:
import math,random
import numpy as np
import pandas as pd
from pyproj import Transformer
from geojson import MultiPoint
import folium
from folium import CircleMarker
from folium.features import GeoJson
from IPython.display import display

def address_info(lng,lat):
    import requests
    import json
    service = "http://api.map.baidu.com/reverse_geocoding/v3/?"
    output = "json"
    AK= "申请的密钥" 
    coordtype = "wgs84ll"
    parameters = f"location={lat},{lng}&output={output}&coordtype={coordtype}&extensions_poi=1&ak={AK}"
    url = service + parameters
    response = requests.get(url)
    text = response.text
    dic = json.loads(text)
    status = dic["status"]
    if status == 0:
        sematic_description = dic["result"]["sematic_description"]
        return sematic_description
    else:
        print("逆向地理编码不成功")      
        
def second_hms(second):
    hour = int(second/3600)
    minute = int((second - 3600*hour)/60)
    second = int(second - 3600*hour - 60*minute)
    return str(hour) + ":" + str(minute) + ":" + str(second)

transformer = Transformer.from_crs(4326,2346,always_xy=True)
input_file = open(r'c:\data\taxi\20150420_26843.txt','r')
min_speed = 5 #定义速度阈值（米/秒）
min_duration = 300 #定义持续时间（秒）
stay_points = []       #停留点列表 
candidate_stays = []   #候选停留列表
i = 0
for record in input_file:
    i = i + 1
    time = record.split(",")[6]
    hour,minute,second = time.split(" ")[1].split(":")
    time_s = int(hour)*3600+int(minute)*60+int(second)
    lng = float(record.split(",")[8])
    lat = float(record.split(",")[9])
    x = transformer.transform(lng,lat)[0]
    y = transformer.transform(lng,lat)[1]
    if i == 1:
        former_time,former_x,former_y = time_s,x,y
    else:
        dtime = time_s - former_time
        dist = math.sqrt((x-former_x)**2+(y-former_y)**2)
        if dtime == 0:
            continue  
        speed = (dist/dtime)
        former_time,former_x,former_y = time_s,x,y
        if speed < min_speed:
            stay_points.append([lng,lat,time_s])
        elif stay_points == []:
            continue
        else:
            candidate_stays.append(stay_points)
            stay_points = []

m = folium.Map([31.3, 121.5], 
               width = 600,height = 400,
               zoom_start=10)
#创建颜色表，用于对每个停留设置不同颜色
import branca.colormap as cm
colormap = cm.LinearColormap(
    colors = ["blue","green","red"],
    vmin=0,vmax=1)
final_stays = []      #最终停留列表
for stay in candidate_stays:
    start_time = stay[0][2]
    end_time = stay[-1][2]
    duration = end_time - start_time
    if duration > min_duration:
        frame = pd.DataFrame(stay,
                           columns=["lng","lat","time"])
        #以下创建GeoJson图层（显示最终停留中所有点）
        coordinates = zip(frame["lng"],frame["lat"])
        multipoint = MultiPoint(list(coordinates))
        color = colormap(random.random())
        circleMarker = CircleMarker(radius=3,
                               weight=6,color=color)
        GeoJson(data=multipoint,marker=circleMarker).add_to(m)
        #以下构建每个最终停留的起止时间、持续时间和地址列表
        center_lng = frame["lng"].mean()
        center_lat = frame["lat"].mean()
        address = address_info(center_lng,center_lat)
        start_time = second_hms(int(start_time))
        end_time = second_hms(int(end_time))
        duration = second_hms(int(duration))
        final_stays.append([start_time,end_time,duration,address])

final_stays = pd.DataFrame(final_stays,
                       columns=["起始时间","结束时间","持续时间","地点"])
display(final_stays)
display(m)

### 示例12-5

In [ ]:
import numpy as np
import geopandas as gpd
from sklearn.cluster import DBSCAN
from pyproj import Transformer
from shapely.geometry import Point
transformer = Transformer.from_crs(4326,2346,always_xy=True)
on_xys = []       #上客点x、y坐标列表
off_xys = []       #上客点x、y坐标列表
on_points = []     #上客点point对象列表
off_points = []     #下客点point对象列表
input_file = open(r'c:\data\taxi\20150420_onOff.txt','r')
for record in input_file:
    on_lng = float(record.split(",")[2])
    on_lat = float(record.split(",")[3])
    on_x = transformer.transform(on_lng,on_lat)[0]
    on_y = transformer.transform(on_lng,on_lat)[1]
    on_xys.append((on_x,on_y))
    on_points.append(Point(on_x,on_y))
    
    off_lng = float(record.split(",")[5])
    off_lat = float(record.split(",")[6])
    off_x = transformer.transform(off_lng,off_lat)[0]
    off_y = transformer.transform(off_lng,off_lat)[1]
    off_xys.append((off_x,off_y))
    off_points.append(Point(off_x,off_y))
input_file.close

dbscan = DBSCAN(eps=1000,min_samples=2000)
#上客点聚类和聚类点输出
on_coords = np.array(on_xys)
clusters = dbscan.fit_predict(on_coords)
gdf = gpd.GeoDataFrame()
gdf["geometry"] = on_points
gdf["class"] = clusters
gdf.crs = "EPSG:2346"
exp = gdf["class"] != -1
gdf[exp].to_file(r"c:\data\tmp\on_cluster.shp") 
#下客点聚类和聚类点输出
off_coords = np.array(off_xys)
clusters = dbscan.fit_predict(off_coords)
gdf = gpd.GeoDataFrame()
gdf["geometry"] = off_points
gdf["class"] = clusters
gdf.crs = "EPSG:2346" 
exp = gdf["class"] != -1
gdf[exp].to_file(r"c:\data\tmp\off_cluster.shp")